# Healthcare Services Location Optimization

## Introduction

In this notebook we aim to analyse a sample of the diagnosed diseases in Texas State and the respective hospitals that treat them. The user can specify various types of diagnosis in the notebook as well. Hospital and patient's data will be collected and an analysis of the cost of the hospital, its distance from the patient and patient demographics is carried out. <br>

In this notebook the user can download real, public patient, hospital and diagnosis data from various websites, transform it into a format for analysis and save the final datasets for visualisation using an Rshiny app. The accelerator already comes packaged with some synthetic data which was created to have similar fields to the publicly available real data. The user can use this data to quickly run through the notebook and visualisation before proceeding with the real data.

To use the real data, change the `data_source` variable to `real`. Use `synthetic` if using the synthetic data packaged with the project. 

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2019, 2020. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**

### Specify the data source

In [1]:
# specify if using data included in project (synthetic) or from the web (real)
# If 'real' is selected, the notebook downloads the data from the sources listed below

data_source = 'synthetic' #synthetic/real

### Install the required libraries

In [2]:
!pip install fuzzywuzzy
!pip install geopy
!pip install python-Levenshtein 

     |████████████████████████████████| 50 kB 1.2 MB/s eta 0:00:011
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=179427 sha256=9cad697bfc9ec4d1bae0397528ac91439c2038b3ac63ad885c62ea30875ac7a6
  Stored in directory: /tmp/1000680000/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [3]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz
from geopy.distance import geodesic
import numpy as np
import os

# project-lib library contains functions that simplify fetching files associated with the project.
from project_lib import Project
project = Project()

## Data that can be downloaded


If the user selects to use the real data, it will be downloaded from the sources listed below and used for the analysis. The user may need to agree to some license terms listed later in this document in order to use this data.<br>

**1: Patients geographical information with latitude, longitude and zip codes   <br>**
Data Source: http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2016_Gazetteer/2016_Gaz_zcta_national.zip  <br>
Column names:  <br>
PAT_ZIP, ALAND, AWATER,ALAND_SQMI, AWATER_SQMI, PAT_LAT,PAT_LONG <br>

**2: Hospital names with longitude and latitude  <br>**
Data Source: https://data.medicare.gov  <br>
Columns:  <br>
Long,	Lat, OBJECTID, NAME, STCTYFIPS, ELEV_METER  <br>


**3:  Diagnosis data in different quarters of the year.  <br>**
Data Source: https://www.dshs.texas.gov  <br>
License Agreement: https://www.dshs.texas.gov/THCIC/Hospitals/Download.shtm <br>
columns:  <br>
DISCHARGE_QTR, THCIC_ID, PROVIDER_NAME, ENCOUNTER_INDICATOR, SEX_CODE, TYPE_OF_ADMISSION, SOURCE_OF_ADMISSION, PAT_STATE, PAT_ZIP, PAT_COUNTRY*  <br>
*Column headers for the above datasets can be found at on https://www.dshs.state.tx.us/thcic/hospitals/UserManual_4q10.pdf 

### Lookups
Below lookup file is used to understand the diagnosis codes. <br>
**1: ICD-9 codes dataset  to match diagnosis  codes:** <br>
http://www.cms.gov/Medicare/Coding/ICD9ProviderDiagnosticCodes/Downloads/ICD-9-CM-v32-master-descriptions.zip

### Get Patients Geographical Information Data

If the user selects to pull the real data, the bash commands below download patient geographical information from the census website. Otherwise, we import a csv file with the synthetic patient geographical information.

In [4]:
if data_source == 'real':
    # Download the data from the  census website and extract
    !wget --quiet  --output-document 2016_Gaz_zcta_national.zip http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2016_Gazetteer/2016_Gaz_zcta_national.zip
    !unzip -o 2016_Gaz_zcta_national.zip -d ZIP/

In [5]:
if data_source == 'real':
    df_patgeo = pd.read_csv("ZIP/2016_Gaz_zcta_national.txt", sep='\t')
    df_patgeo.columns = df_patgeo.columns.str.strip().str.upper().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df_patgeo= df_patgeo.rename(columns={"GEOID": 'PAT_ZIP', "INTPTLAT": 'PAT_LAT', "INTPTLONG": 'PAT_LONG'})

elif data_source == 'synthetic':
    

    zip_location_data_file = project.get_file("synthetic patients zip location data.csv")
    zip_location_data_file.seek(0)
    df_patgeo = pd.read_csv(zip_location_data_file)

df_patgeo = df_patgeo[['PAT_ZIP', 'PAT_LAT', 'PAT_LONG']]
df_patgeo.head()

,PAT_ZIP,PAT_LAT,PAT_LONG
0,73949,36.563724,-101.895458
1,75001,32.895282,-96.857488
2,75002,33.086085,-96.639902
3,75006,32.917130,-96.834502
4,75007,32.909405,-96.839330


Patients geographical data contains following columns

**PAT_ZIP :** Patients zip code. <br>
**PAT_LAT :** Latitude corresponding to the zip code. <br>
**PAT_LONG :** Longitude corresponding to the zip code. <br>

### Get Hospital information data 

The data from the website https://data.medicare.gov/api/views/xubh-q36u/rows.csv?accessType=DOWNLOAD&sorting=true can be downloaded. If the synthetic data is being used instead, it is imported from a csv file using `project_lib` library. The file was created to have similar fields and content to the real data.<br> 
The dataset contains details on hospitals and their locations.

In [6]:
if data_source == 'real':
    # Below bash command will download the data and that will be stored in a dataframe dfHospital
    z=!wget -O hospital.csv https://data.medicare.gov/api/views/xubh-q36u/rows.csv?accessType=DOWNLOAD&sorting=true 
    dfHospital = pd.read_csv("hospital.csv", sep=',')
    # dfHospital dataset is cleaned to retrieve only texas state records
    dfHospital=dfHospital[dfHospital['State']=='TX']
elif data_source == 'synthetic':
    hospital_details_file = project.get_file("synthetic hospital details.csv")
    hospital_details_file.seek(0)
    dfHospital = pd.read_csv(hospital_details_file)

Extract longitude and latitude from the location.

Hospital dataset contains below columns <br>

**Hospital_Name:** Name of the hospital <br>
**longitude & latitude:** Coordinates of the hospital <br>

In [7]:
# seperate longitude and latitude columns and remove the additional characters
if data_source == 'real':
    dfHospital['longlat']=dfHospital['Location'].str.split("(").str[1].str.strip(')')
    dfHospital['longitude']=dfHospital['longlat'].str.split(' ').str[0]
    dfHospital['latitude']=dfHospital['longlat'].str.split(' ').str[1]
elif data_source == 'synthetic':
    dfHospital['longlat']=dfHospital['Location'].str.split("(").str[1].str.strip(')')
    dfHospital['longitude']=dfHospital['longlat'].str.split(',').str[1]
    dfHospital['latitude']=dfHospital['longlat'].str.split(',').str[0]
    
# Rename the columns 
dfHospital=dfHospital[['Facility ID','Facility Name','longitude','latitude']]
dfHospital.columns=['Provider_ID','Hospital_Name','longitude','latitude']
#Remove the columns with nulls
dfHospital=dfHospital[pd.notnull(dfHospital['longitude'])]
dfHospital.head()

,Provider_ID,Hospital_Name,longitude,latitude
0,450558,Abilene Regional Medical Center,-99.746965,32.42423
1,670102,"Ad Hospital East, Llc",-95.386342,29.780471999999996
2,450488,Allegiance Specialty Hospital Of Kilgore,-94.859589,32.397809
3,670109,Altus Baytown Hospital,-94.967955,29.756974
4,450078,Anson General Hospital,-99.896175,32.754511


### Get Patient Diagnosis data

The code below contains the link to an example of real world diagnosis data that can be used. The user should read and agree to the license terms at the following link before using the data. https://www.dshs.texas.gov/THCIC/Hospitals/Download.shtm. The user can then download the data themselves or uncomment and use the code in the cell below. The cells take a few minutes to run as the files downloaded are quite large.  

> The bash commands specified below are used to download the data. <br>
> Specify the quarter and year of the data that to be used for the analysis purpose. <br>
> Below set of code downloads the data for the years between 2008 and 2010. Once downloaded, these files will be extracted into ZIP folder. <br>
> Each PUDF_base*.txt file is processed and stored into a dataframe. Once the dataframe is loaded, all the files and zip folder will be deleted <br>

In [8]:
%%bash -s "$data_source"
if [ "$1" = "real" ]; then  # Download data from the above website, specify year and the diagnosis code

years=(1Q2008 2Q2008 3Q2008 4Q2008 1Q2009 2Q2009 3Q2009 4Q2009 1Q2010 2Q2010 3Q2010 4Q2010)
for year in "${years[@]}";
do
wget -O filename$year.zip https://www.dshs.texas.gov/thcic/hospitals/Data/PUDF-"$year"-tab-delimited/ --no-check-certificate --quiet
done
fi

In [9]:
if data_source == 'real':
    # Unzip the files and remove the unnecessary files from the disk, Once all the files are extracted remove the zip files too
    !unzip 'filename*.zip' -d ZIP/
    !rm -rf out*
    !rm -rf *.zip
    !rm -rf PUDF_charges*

After all of the files are downloaded we can process the PUDF_base*.txt files. <br>

Enter the diagnosis codes required for the user to analyse. Diagnosis codes can be found on the website **https://www.cms.gov/**
These diagnosis codes are in ICD-9 format.  International Classification of Diseases, Clinical Modification (ICD-9-CM) is an adaption created by the U.S. National Center for Health Statistics (NCHS) and used in assigning diagnostic and procedure codes associated with inpatient, outpatient, and physician office utilization in the United States.

We have used the diagnosis codes <b>[7453, 74510, 74569, 74511, 7450]</b> for this analysis. These diagnosis are cardiac related, which are usually found in children. (https://en.wikipedia.org/wiki/Bulbus_cordis)

In [10]:
if data_source == 'real':
    diagnosis_codes=["7453","74510","74569","74511","7450"]
    data=pd.DataFrame()

    columns= ['DISCHARGE_QTR', 'THCIC_ID', 'PROVIDER_NAME', 'SPEC_UNIT','ENCOUNTER_INDICATOR', 'SEX_CODE', 'TYPE_OF_ADMISSION','SOURCE_OF_ADMISSION', 'PAT_STATE', 'PAT_ZIP', 'PAT_COUNTRY', 'COUNTY','PUBLIC_HEALTH_REGION', 'ADMIT_WEEKDAY', 'LENGTH_OF_STAY', 'PAT_AGE','PAT_STATUS', 'RACE', 'ETHNICITY', 'FIRST_PAYMENT_SRC','SECONDARY_PAYMENT_SRC', 'TYPE_OF_BILL', 'PRIVATE_AMOUNT','CLINIC_AMOUNT', 'TOTAL_CHARGES', 'ADMITTING_DIAGNOSIS','PRINC_DIAG_CODE', 'PRINC_ICD9_CODE', 'RISK_MORTALITY','ILLNESS_SEVERITY', 'ATTENDING_PHYS_UNIF_ID', 'OPERATING_PHYS_UNIF_ID']
    dtypes={'DISCHARGE_QTR':object,'THCIC_ID':int,'PROVIDER_NAME':object,'SPEC_UNIT': object,'ENCOUNTER_INDICATOR':  int,'SEX_CODE':  object,'TYPE_OF_ADMISSION':object,'SOURCE_OF_ADMISSION': object,'PAT_STATE': object,'PAT_ZIP':object,'PAT_COUNTRY':  object,'COUNTY':float,'PUBLIC_HEALTH_REGION':  float,'ADMIT_WEEKDAY': int,'LENGTH_OF_STAY':  float,'PAT_AGE':object,'PAT_STATUS':object,'RACE':object,'ETHNICITY': object,'FIRST_PAYMENT_SRC':object,'SECONDARY_PAYMENT_SRC':  object,'TYPE_OF_BILL': object,'PRIVATE_AMOUNT':  float,'CLINIC_AMOUNT':float,'TOTAL_CHARGES':float,'ADMITTING_DIAGNOSIS': object,'PRINC_DIAG_CODE':  object,'PRINC_ICD9_CODE':  object,'RISK_MORTALITY':int,'ILLNESS_SEVERITY':  int,'ATTENDING_PHYS_UNIF_ID':float,'OPERATING_PHYS_UNIF_ID':float}

    # for each file in the zip folder identify the filenames start with PUDF_base* and read the file into a dataframe 
    for root, dirs, files in os.walk("ZIP"):
        for name in files:
            if("PUDF_base" in name):
                print("Reading file ",name)
                df = pd.read_csv(root+"/"+name,delimiter='\t',usecols = columns,dtype=dtypes)
                data=data.append(df[df['PRINC_DIAG_CODE'].isin(diagnosis_codes)])
    data=data.rename(columns={'THCIC_ID':'PROVIDER_ID'})

If the user has selected to run the notebook with the synthetic data, the diagnosis data is imported from the csv file. The data contains 5 different diagnosis codes, 7453, 74510, 74569, 74511 and 7450.

In [11]:
if data_source == 'synthetic':
    diagnosis_data_file = project.get_file("synthetic diagnosis data.csv")
    diagnosis_data_file.seek(0)
    data = pd.read_csv(diagnosis_data_file)

Once the data has been read into the variable, the next step is to find the hospitals with the highest number of patients. We take the top 15 number of hospitals and filter out all the remaining data from the dataset. 

In [12]:
# Find the count of patients in each hospital 
dfDIAGCODE_count=data.groupby(["PROVIDER_NAME"]).count()['DISCHARGE_QTR'].reset_index()
# Sort the dataframe by count 
dfDIAGCODE_top=dfDIAGCODE_count.sort_values('DISCHARGE_QTR', ascending=False)
# Select top 15 hospital names and store them in a list top_hospitals
top_hospitals=dfDIAGCODE_top.head(15).PROVIDER_NAME.tolist()
# filter the data with the hospital names in the list top_hospitals
dfDIAGCODE_pd=data[data.PROVIDER_NAME.isin(top_hospitals)].copy()
dfDIAGCODE_pd.head()

,ENCOUNTER_Id,ENCOUNTER_START,ENCOUNTER_STOP,ENCOUNTER_BASE_ENCOUNTER_COST,TOTAL_CHARGES,ENCOUNTER_PAYER_COVERAGE,PAT_BIRTHDATE,ETHNICITY,RACE,SEX_CODE,PAT_STATE,PAT_ZIP,PROVIDER_ID,PROVIDER_NAME,FIRST_PAYMENT_SRC,PAT_AGE,ADMIT_DAY_OF_WEEK,LENGTH_OF_STAY,DISCHARGE_QTR,PRINC_DIAG_CODE
0,e1ba4c20-f02f-4e6b-8606-5675e498cff8,2019-06-14 22:16:01,2019-07-08 22:16:01,2559.36,823270.336041,1119.36,2018-01-16 00:00:00,hispanic,white,F,Texas,78264,48773,Methodist Stone Oak Hospital,Cigna Health,1,4,24,2019Q3,74569
1,eb6a3598-e810-4f0b-99e9-4f7404076fbc,2016-12-31 07:06:31,2017-01-01 07:06:31,106.64,33360.837508,0.00,2016-05-05 00:00:00,nonhispanic,white,F,Texas,78226,48773,Methodist Stone Oak Hospital,NO_INSURANCE,1,5,1,2017Q1,74510
2,c4ff6819-dc78-46f1-92c7-b6caa2deb00f,2017-01-13 07:06:31,2017-01-23 07:06:31,1066.40,83175.186343,0.00,2016-05-05 00:00:00,nonhispanic,white,F,Texas,78226,48773,Methodist Stone Oak Hospital,NO_INSURANCE,1,4,10,2017Q1,74510
3,5dc4bd2d-ad88-44c0-a756-f45110785172,2018-05-12 07:06:31,2018-05-17 07:06:31,533.20,28449.654727,0.00,2016-05-05 00:00:00,nonhispanic,white,F,Texas,78226,48773,Methodist Stone Oak Hospital,NO_INSURANCE,2,5,5,2018Q2,74569
4,4662838b-a406-4f23-a591-7d99fe2754ea,2018-11-08 07:06:31,2018-11-10 07:06:31,213.28,38822.072370,0.00,2016-05-05 00:00:00,nonhispanic,white,F,Texas,78226,48773,Methodist Stone Oak Hospital,NO_INSURANCE,3,3,2,2018Q4,7453


In [13]:
if data_source == 'real':
    # Delete the files once dataset is loaded
    !rm -rf ZIP

The dataset downloaded above includes all the data related to the condition selected by the user. The above dataset contains below columns.<br>
<br>**DISCHARGE_QTR:** Quarter and year in which patient discharged
<br>**PROVIDER_ID:** Hospital id.
<br>**PROVIDER_NAME:** Hospital Name.
<br>**SEX_CODE:** Sex of the patient admitted.
<br>**PAT_STATE:** State in which patient resides.
<br>**PAT_ZIP:** Patients zip code
<br>**LENGTH_OF_STAY:** Length in days patient admitted in the hospital
<br>**PAT_AGE:** Patient's age
<br>**RACE:** Patient's Race
<br>**TOTAL_CHARGES:** Total Charges for the treatment
<br>**PRINC_DIAG_CODE:** Diagnosis code (7453 for common ventricle disease)

In [14]:
dfDIAGCODE_pd.tail()

,ENCOUNTER_Id,ENCOUNTER_START,ENCOUNTER_STOP,ENCOUNTER_BASE_ENCOUNTER_COST,TOTAL_CHARGES,ENCOUNTER_PAYER_COVERAGE,PAT_BIRTHDATE,ETHNICITY,RACE,SEX_CODE,PAT_STATE,PAT_ZIP,PROVIDER_ID,PROVIDER_NAME,FIRST_PAYMENT_SRC,PAT_AGE,ADMIT_DAY_OF_WEEK,LENGTH_OF_STAY,DISCHARGE_QTR,PRINC_DIAG_CODE
44862,e135b847-ee18-4782-85dc-55935c05cb03,2018-04-12 21:24:12,2018-04-14 21:24:12,213.28,48957.090208,93.28,2016-02-07 00:00:00,nonhispanic,asian,M,Texas,75057,735d1,Medical Center Of Lewisville,Cigna Health,2,3,2,2018Q2,74569
44863,5514ff66-49e1-42c0-bfb6-544605b69cb3,2019-02-13 21:24:12,2019-03-10 21:24:12,2666.00,455227.675977,1166.00,2016-02-07 00:00:00,nonhispanic,asian,M,Texas,75057,735d1,Medical Center Of Lewisville,Cigna Health,3,2,25,2019Q1,74569
44864,794ee761-b9d5-45e9-89aa-5cd8b32020a6,2018-08-21 17:50:40,2018-09-14 17:50:40,2559.36,210858.865666,0.00,2018-05-21 00:00:00,nonhispanic,white,F,Texas,75057,735d1,Medical Center Of Lewisville,NO_INSURANCE,0,1,24,2018Q3,74569
44865,bed48d03-1a05-4459-ab97-10605c361eb7,2019-11-08 12:42:25,2019-11-13 12:42:25,533.20,113934.800581,208.20,2016-09-23 00:00:00,nonhispanic,white,F,Texas,77571,735d1,Medical Center Of Lewisville,Aetna,3,4,5,2019Q4,74569
44866,bfa49b85-c36e-45c9-898c-920565b72355,2018-03-23 10:13:48,2018-03-27 10:13:48,426.56,141742.586347,186.56,2017-09-22 00:00:00,hispanic,white,M,Texas,78363,735d1,Medical Center Of Lewisville,Cigna Health,0,4,4,2018Q1,74569


## Data Cleaning and Prep

If using the real data, the payment source and age group are coded and need to be mapped:
    1. Create a dictionary lookup based on the manual https://www.dshs.state.tx.us/thcic/hospitals/UserManual_4q10.pdf for the column FIRST_PAYMENT_SRC.
    2. Create a dictionary lookup based on the manual https://www.dshs.state.tx.us/thcic/hospitals/UserManual_4q10.pdf for the column PRICE_DIAG_CODE.
    
Finally, create a dictionary lookup for the column PRINC_DIAG_CODE. The information is available on http://icd9.chrisendres.com/index.php?action=child&recordid=7440.

In [15]:
# if using thcic data the user needs to map payment and age information. 
# in the synthetic data, this is already done
if data_source == 'real':
    paymentsrc_di={"09": "Self Pay", 
    "HM":"Health Maintenance Organization",
    "10": "Central Certification",
    "LI": "Liability",
    "11": "Other Non-federal Programs" ,
    "LM": "Liability Medical",
    "12": "Preferred Provider Organization (PPO)",
    "MA": "Medicare Part A",
    "13": "Point of Service (POS)", 
    "MB": "Medicare Part B",
    "14": "Exclusive Provider Organization (EPO)",
    "MC": "Medicaid",
    "15": "Indemnity Insurance", 
    "TV": "Title V",
    "16": "Health Maintenance Organization (HMO)",
    "OF": "Other Federal Program",
    "AM": "Automobile Medical",
    "VA": "Veteran Administration Plan",
    "BL": "Blue Cross/Blue Shield", 
    "WC": "Workers Compensation Health Claim",
    "CH": "CHAMPUS", 
    "ZZ": "Charity, Indigent",
    "CI": "Commercial Insurance", 
    "DS": "Disability Insurance", 
    "*":"Invalid",
    "**":"Invalid"}

    #map age group 
    age_di={
    "00":"001-028 days",
    "01":"029-365 days",
    "02":"01-04",
    "03":"05-09",
    "04":"10-14",
    "05":"15-17",
    "06":"18-19",
    "07":"20-24",
    "08":"25-29",
    "09":"30-34",
    "10":"35-39",
    "11":"40-44",
    "12":"45-49",
    "13":"50-54",
    "14":"55-59",
    "15":"60-64",
    "16":"65-69",
    "17":"70-74",
    "18":"75-79",
    "19":"80-84",
    "20":"85-89",
    "21":"90+",
    "*":"Invalid"}

In [16]:
# map icd9 code into diagnosis name
PRINC_DIAG_di={"74569":"74569-Endocard cushion def",
"74511":"74511-Double outlet right ventricle",
"74510":"74510-Complete transp of great vessels",
"7453":"7453-Common ventricle",
"7450":"7450-Common truncus"}

The above dictionaries are used to map the columns in the final datasets 

3 data sets are used for this analysis. <br>

**dfDIAGCODE_pd** : Patient diagnosis information <br>
**dfHospital**: Hospitals location information <br>
**df_patgeo**: Patient location information <br>

Data is prepared and cleaned in sequence of steps to join these data sets and then to calculate the followings.
1. Prepare dfDIAGCODE_pd dataset.
2. Prepare dfHospital dataset.
3. Find the coordinates of the hospitals using dfHospital and join it with dfDIAGCODE_pd using fuzzy match.
4. Find the coordinates of patients using df_patgeo.
5. Calculate the distance between the patient and the hospital that they visited for the treatment using the Levenstein distance. (The Levenshtein distance is a string metric for measuring the difference between two sequences.)
6. Calculate the distance between the patient and the nearest hospital.
7. Check if the hospital the patient visited is same as the nearest hospital.
8. Visualize the final data in terms of distance, cost and severity of the illness etc. 




#### Hospitals that treat this condition

In [17]:
print(dfDIAGCODE_pd.PROVIDER_NAME.drop_duplicates().to_string(index=False))

                      Methodist Stone Oak Hospital
                  St Joseph Regional Health Center
                           Pearland Medical Center
                   Baylor Emergency Medical Center
                   Mission Regional Medical Center
       Baylor Scott & White Medical Center Garland
                            Texas General Hospital
 Baylor Scott And White Medical Center Lake Pointe
                            Baptist Medical Center
                   Baylor Medical Center At Irving
                        Memorial Hermann Northeast
                          Resolute Health Hospital
                 Crescent Medical Center Lancaster
              First Texas Hospital Carrrollton Llc
                      Medical Center Of Lewisville


 #### Diagnosis data cleaning

In [18]:
# Diagnosis data cleaning
dfDIAGCODE_pd['PAT_ZIP'] = pd.to_numeric(dfDIAGCODE_pd['PAT_ZIP'],errors='coerce')
dfDIAGCODE_pd = dfDIAGCODE_pd.dropna(subset=['PAT_ZIP'])
dfDIAGCODE_pd['PAT_ZIP'] = dfDIAGCODE_pd['PAT_ZIP'].astype(int)

In [19]:
dfDIAGCODE_pd['PROVIDER_ID'] = dfDIAGCODE_pd['PROVIDER_ID'].astype(str)
dfDIAGCODE_pd['RACE'] = dfDIAGCODE_pd['RACE'].astype(str)
dfDIAGCODE_pd['ETHNICITY'] = dfDIAGCODE_pd['ETHNICITY'].astype(str)

# Seperate year and quarter into seperate columns , and find charges per day based on total charges and length of stay
dfDIAGCODE_pd['YEAR'] = dfDIAGCODE_pd.DISCHARGE_QTR.str.slice(0,4)
dfDIAGCODE_pd['QTR'] = dfDIAGCODE_pd.DISCHARGE_QTR.str.slice(5,6)
dfDIAGCODE_pd['CHARGES_PER_DAY'] = dfDIAGCODE_pd['TOTAL_CHARGES'] / dfDIAGCODE_pd['LENGTH_OF_STAY'] 

# do some additional cleaning on the thcic data
if data_source == 'real':
    dfDIAGCODE_pd['PROVIDER_NAME'] = dfDIAGCODE_pd['PROVIDER_NAME'].str.strip()
    dfDIAGCODE_pd['PROVIDER_NAME'] = dfDIAGCODE_pd['PROVIDER_NAME'].str.replace('-',' at ')
    dfDIAGCODE_pd['PROVIDER_NAME'] = dfDIAGCODE_pd['PROVIDER_NAME'].str.replace('Rehab','Rehabilitation')
    dfDIAGCODE_pd['PROVIDER_NAME'] = dfDIAGCODE_pd['PROVIDER_NAME'].str.replace('UT ','University of Texas ') 

In [20]:
if data_source == 'real':
    # Update columns based on the above dictionaries
    # 1. Update FIRST_PAYMENT_SRC with paymentsrc_di - replacing codes wit names
    dfDIAGCODE_pd['FIRST_PAYMENT_SRC']=dfDIAGCODE_pd['FIRST_PAYMENT_SRC'].map(paymentsrc_di)
    # 2. AGE_GROUP from PAT_AGE and age_di
    dfDIAGCODE_pd['PAT_AGE']=dfDIAGCODE_pd['PAT_AGE'].map(age_di)

#3. PRINC_DIAG_DESC from PRINC_DIAG_CODE and PRINC_DIAG_di - do this for all data
dfDIAGCODE_pd['PRINC_DIAG_DESC']=dfDIAGCODE_pd['PRINC_DIAG_CODE'].astype(str).map(PRINC_DIAG_di)

###  Hospital Data cleaning

#### Match hospitals names

Above two datasets contain various hospital names. Depending on the source of the data, it is possible that some hospitals have different names in the different datasets. We used fuzzy match from fuzzywuzzy library to join these two datasets. It uses Levenshtein Distance to calculate the differences between sequences.

In [21]:

# Look for the providers name in the hospital data to fetch the longitude and latitude for the data 
# Create an index by providing provider_id, longitude and latitude from dfHospital dataset
choices = dfHospital.set_index(["Provider_ID",'longitude','latitude']).Hospital_Name.to_dict()

# Look for the hospital names in the dfDIAGCODE_pd to find a matching hospital, the returning set contains Provider_name from dfDIAGCODE_pd, Hospital_Name from dfHospital, matching score and the index as an ID
res = [(lookup,) + item for lookup in dfDIAGCODE_pd['PROVIDER_NAME'].unique() for item in process.extract(lookup, choices,limit=1)]
dfHospitalsMatch = pd.DataFrame(res, columns=["PROVIDER_NAME", "Hospital_Name_From_hosp", "Matching_Score", "Combine_ID"])

In [22]:
dfHospitalsMatch

,PROVIDER_NAME,Hospital_Name_From_hosp,Matching_Score,Combine_ID
0,Methodist Stone Oak Hospital,Methodist Stone Oak Hospital,100,"(670055, -98.525142, 29.472403000000003)"
1,St Joseph Regional Health Center,St Joseph Regional Health Center,100,"(450011, -96.46521899999999, 30.714431)"
2,Pearland Medical Center,Pearland Medical Center,100,"(670106, -95.32307, 29.555760999999997)"
3,Baylor Emergency Medical Center,Baylor Emergency Medical Center,100,"(670062, -96.9826, 33.302002)"
4,Mission Regional Medical Center,Mission Regional Medical Center,100,"(450176, -95.666338, 29.693693)"
5,Baylor Scott & White Medical Center Garland,Baylor Scott & White Medical Center Garland,100,"(450280, -96.63035699999999, 32.909826)"
6,Texas General Hospital,Texas General Hospital,100,"(670083, -97.020995, 32.684193)"
7,Baylor Scott And White Medical Center Lake Pointe,Baylor Scott And White Medical Center Lake Pointe,100,"(450742, -96.548673, 32.915921999999995)"
8,Baptist Medical Center,Baptist Medical Center,100,"(450058, -98.531182, 29.233462)"
9,Baylor Medical Center At Irving,Baylor Medical Center At Irving,100,"(450079, -96.970022, 32.857748)"


We decided to exclude the hospital names that have a Levenshtein Distance less than 86.

In [23]:
dfHospitalsMatch=dfHospitalsMatch[dfHospitalsMatch['Matching_Score']>86].copy()
dfHospitalsMatch['PVDR_LONG']=dfHospitalsMatch['Combine_ID'].astype(str).str.split(',').str[1].str.replace("'","")
dfHospitalsMatch['PVDR_LAT']=dfHospitalsMatch['Combine_ID'].astype(str).str.split(',').str[2]
dfHospitalsMatch['PVDR_LAT']=dfHospitalsMatch['PVDR_LAT'].str.replace(")","").str.replace("'","")
dfHospitalsMatch

,PROVIDER_NAME,Hospital_Name_From_hosp,Matching_Score,Combine_ID,PVDR_LONG,PVDR_LAT
0,Methodist Stone Oak Hospital,Methodist Stone Oak Hospital,100,"(670055, -98.525142, 29.472403000000003)",-98.525142,29.472403000000003
1,St Joseph Regional Health Center,St Joseph Regional Health Center,100,"(450011, -96.46521899999999, 30.714431)",-96.46521899999999,30.714431
2,Pearland Medical Center,Pearland Medical Center,100,"(670106, -95.32307, 29.555760999999997)",-95.32307,29.555760999999997
3,Baylor Emergency Medical Center,Baylor Emergency Medical Center,100,"(670062, -96.9826, 33.302002)",-96.9826,33.302002
4,Mission Regional Medical Center,Mission Regional Medical Center,100,"(450176, -95.666338, 29.693693)",-95.666338,29.693693
5,Baylor Scott & White Medical Center Garland,Baylor Scott & White Medical Center Garland,100,"(450280, -96.63035699999999, 32.909826)",-96.63035699999999,32.909826
6,Texas General Hospital,Texas General Hospital,100,"(670083, -97.020995, 32.684193)",-97.020995,32.684193
7,Baylor Scott And White Medical Center Lake Pointe,Baylor Scott And White Medical Center Lake Pointe,100,"(450742, -96.548673, 32.915921999999995)",-96.548673,32.915921999999995
8,Baptist Medical Center,Baptist Medical Center,100,"(450058, -98.531182, 29.233462)",-98.531182,29.233462
9,Baylor Medical Center At Irving,Baylor Medical Center At Irving,100,"(450079, -96.970022, 32.857748)",-96.970022,32.857748


In [24]:
hospitalTX=dfHospitalsMatch[['PROVIDER_NAME','PVDR_LAT','PVDR_LONG']]

In [25]:
## Add longitude and latitude to the patients zipcode
hospitalTX

,PROVIDER_NAME,PVDR_LAT,PVDR_LONG
0,Methodist Stone Oak Hospital,29.472403000000003,-98.525142
1,St Joseph Regional Health Center,30.714431,-96.46521899999999
2,Pearland Medical Center,29.555760999999997,-95.32307
3,Baylor Emergency Medical Center,33.302002,-96.9826
4,Mission Regional Medical Center,29.693693,-95.666338
5,Baylor Scott & White Medical Center Garland,32.909826,-96.63035699999999
6,Texas General Hospital,32.684193,-97.020995
7,Baylor Scott And White Medical Center Lake Pointe,32.915921999999995,-96.548673
8,Baptist Medical Center,29.233462,-98.531182
9,Baylor Medical Center At Irving,32.857748,-96.970022


The above dataset now contains the hospital names with longitude and latitude in it.

# Find nearest hospital and distance 
 This function performs following tasks
 1. Assigns latitude and longitude to each patient based on PAT_ZIP.
 2. Calculates the distance travelled by patient to the hospital using longitude and latitudes.
 3. Calculates distance to all the hospitals that treat this condition.
 4. Find the nearest hospital and distance to it.
 5. Checks if the nearest hospital is same as the one patient visited.
 6. Calculates extra distance travelled by the patient.

In [26]:
def find_closest_hospital(df_patgeo,dfDIAGCODE_pd,hospitalTX):
       # Patients data belongs to Texas
    df1=df_patgeo[((df_patgeo['PAT_ZIP'] > 73000) & (df_patgeo['PAT_ZIP'] < 80000)) | ((df_patgeo['PAT_ZIP'] > 88000) & (df_patgeo['PAT_ZIP'] < 89000))] .copy()
    # Copy of diagnosis data 
    df2 = dfDIAGCODE_pd.copy()

    # Create a lookupmatrix to assign longitude and latitude to each record with PAT_ZIP
    dfLookupDistance = pd.merge(df1, df2, how='right',on=['PAT_ZIP'])
    #dfLookupDistance.drop('tmp',1)

    # New dataframe by joining hospital data and lookup data on provider name
    dfTXPTNT2Hospitals = pd.merge(hospitalTX, dfLookupDistance, how = 'left', on='PROVIDER_NAME')

    # Drop nulls 
    dfTXPTNT2Hospitals = dfTXPTNT2Hospitals.dropna(subset=['PVDR_LAT'])
    dfTXPTNT2Hospitals = dfTXPTNT2Hospitals.dropna(subset=['PVDR_LONG'])
    dfTXPTNT2Hospitals = dfTXPTNT2Hospitals.dropna(subset=['PAT_LAT'])
    dfTXPTNT2Hospitals = dfTXPTNT2Hospitals.dropna(subset=['PAT_LONG'])

    # Calculate distance between the patient and the hospital they were admitted to 
    dfTXPTNT2Hospitals['DISTANCE'] = dfTXPTNT2Hospitals.apply(lambda x: geodesic((x['PVDR_LAT'], x['PVDR_LONG']), (x['PAT_LAT'], x['PAT_LONG'])).miles, axis = 1)
    Diagdata=dfTXPTNT2Hospitals.copy()
    hospitaldata = hospitalTX.copy()
    # Cross Join Diagnosis data and hospital data to get all possible combinations
    Diagdata['tmp']=1
    hospitaldata['tmp']=1
    
    crs_data=pd.merge(Diagdata,hospitaldata,on='tmp')
    
    # Create a new column to calculate distance to all the hospitals using longitude and latitude
    crs_data['DISTANCE_TO_HOSPITAL'] = crs_data.apply(lambda x: geodesic((x['PVDR_LAT_y'], x['PVDR_LONG_y']), (x['PAT_LAT'], x['PAT_LONG'])).miles, axis = 1)
    
    # Check the hospital visited by the patient is nearest or not
    crs_data['IS_NEAREST'] = crs_data.groupby(['PAT_ZIP'])['DISTANCE_TO_HOSPITAL'].transform(min) == crs_data['DISTANCE_TO_HOSPITAL']
    
    # Find the distance to the nearest hospital
    crs_data['MIN_DISTANCE']=crs_data.groupby(['PAT_ZIP'])['DISTANCE_TO_HOSPITAL'].transform(min)
    
    # Create a dataset with the nearest hospital only
    PROV2ZIP = crs_data.loc[crs_data.groupby('PAT_ZIP')['DISTANCE_TO_HOSPITAL'].idxmin()]
    MIN_PROV2ZIP = pd.DataFrame(data=PROV2ZIP,columns=['PAT_ZIP','PROVIDER_NAME_y'])
    MIN_PROV2ZIP= MIN_PROV2ZIP.rename(columns={"PROVIDER_NAME_y": "NEAREST_HOSPITAL"})
    
    
    # Create another dataset with unique combination of minimum distance and join it with crossjoin data
    NEAREST_HOSPITAL_DISTANCE = crs_data.groupby('PAT_ZIP')["DISTANCE_TO_HOSPITAL"].min().rename("NEAREST_HOSPITAL_DISTANCE").reset_index()
    allDist_PP_0 = crs_data.merge(NEAREST_HOSPITAL_DISTANCE)
    
    # Join above two datasets on PAT_ZIP
    allDist_PP_1 = pd.merge(allDist_PP_0, MIN_PROV2ZIP, how = 'left', on='PAT_ZIP')
    
    #Create a new column 'TRADEOFF' to calculate extra distance travelled by the patient
    allDist_PP_1['TRADEOFF'] = allDist_PP_1['DISTANCE_TO_HOSPITAL'] - allDist_PP_1['NEAREST_HOSPITAL_DISTANCE']
    
    allDist_PP_1=allDist_PP_1[round(allDist_PP_1['DISTANCE'],3)==round(allDist_PP_1['DISTANCE_TO_HOSPITAL'],3)]
    #Remove duplicates
    allDist_PP_1= allDist_PP_1.drop_duplicates()
    # Create a dataset with selected columns
    Tradeoff_matrix = allDist_PP_1[['PAT_ZIP','PROVIDER_NAME_x', 'DISTANCE_TO_HOSPITAL', 'IS_NEAREST','NEAREST_HOSPITAL_DISTANCE','NEAREST_HOSPITAL','TRADEOFF','PVDR_LONG_x','PVDR_LAT_x','PAT_LONG','PAT_LAT']].copy()
    Tradeoff_matrix['PROVIDER_NAME'] = Tradeoff_matrix['PROVIDER_NAME_x'].astype(str)
    
    #Create a final datasets by 
    AllPatients = pd.merge(dfDIAGCODE_pd, Tradeoff_matrix, how = 'left', on=['PAT_ZIP','PROVIDER_NAME']) 
    AllPatients=AllPatients.drop_duplicates()
    AllPatients = AllPatients[pd.notnull(AllPatients['NEAREST_HOSPITAL_DISTANCE'])]
    AllPatients=AllPatients.rename(columns={"PVDR_LONG_x": "PVDR_LONG", "PVDR_LAT_x": "PVDR_LAT"})
    return AllPatients

In [27]:
# Create a blank dataframe
AllPatients=pd.DataFrame()

# For each diagnosis code in dfDIAGCODE_pd call the function find_closest_hospital() and append it to the dataframe
for code in dfDIAGCODE_pd["PRINC_DIAG_CODE"].unique().tolist():
    AllPatients=AllPatients.append(find_closest_hospital(df_patgeo,dfDIAGCODE_pd[dfDIAGCODE_pd["PRINC_DIAG_CODE"]==code],hospitalTX))

The patient counts for each diagnosis code

In [28]:
dfDIAGCODE_pd["PRINC_DIAG_CODE"].value_counts()



74569    3292
74511    2009
74510    1511
7453     1276
7450     1008
Name: PRINC_DIAG_CODE, dtype: int64

The final data set contains the information on the hospitals for the diagnosis selected by the users. It includes patient's personal details, distance travelled by the patients to the hospital, nearest hospital to the patient, cost of hospital stay, length of stay before discharging and total charges.

The final dataset contains below columns.

<br>**DISCHARGE_QTR:** Quarter and year in which patient discharged
<br>**PROVIDER_ID:** Hospital id.
<br>**PROVIDER_NAME:** Hospital Name.
<br>**PVDR_LONG:** Hospital Longitude
<br>**PVDR_LAT:** Hospital Latitude
<br>**PAT_LONG:** Patient Longitude
<br>**PAT_LAT:** Patient Latitude
<br>**SEX_CODE:** Sex of the patient admitted.
<br>**PAT_STATE:** State in which patient resides.
<br>**PAT_ZIP:** Patient's zip code
<br>**LENGTH_OF_STAY:** Length in days patient stayed in the hospital
<br>**PAT_AGE:** Patient's age
<br>**RACE:** Patient's Race
<br>**TOTAL_CHARGES:** Total Charges for the treatment
<br>**PRINC_DIAG_CODE:** Principle Diagnosis code 
<br>**CHARGES_PER_DAY:** Patient charges per day
<br>**DISTANCE_TO_HOSPITAL:** Distance travelled by the patient to the hospital
<br>**NEAREST_HOSPITAL:** Name of the nearest hospital
<br>**IS_NEAREST:** Boolean variable to identify if the Hospital visited by the patient is nearest.
<br>**NEAREST_HOSPITAL_DISTANCE:** Distance to the nearest hospital.
<br>**TRADEOFF:** Additional distance travelled by the patient.
<br>**FIRST_PAYMENT_SRC:** Payment source description.
<br>**PAT_AGE:** PAT_AGE group
<br>**PRINC_DIAG_DESC:** Descriptions for PRINC_DIAG_CODE.

### Save the dataset to build dashboard

In [29]:
project.save_data("AllPatients.csv", AllPatients.to_csv(index=False), overwrite=True)

{'file_name': 'AllPatients.csv',
 'message': 'File saved to project storage.',
 'asset_id': '9169f91d-897b-4444-ba6b-b347f2387f07'}

In [30]:
AllPatients.columns

Index(['ENCOUNTER_Id', 'ENCOUNTER_START', 'ENCOUNTER_STOP',
       'ENCOUNTER_BASE_ENCOUNTER_COST', 'TOTAL_CHARGES',
       'ENCOUNTER_PAYER_COVERAGE', 'PAT_BIRTHDATE', 'ETHNICITY', 'RACE',
       'SEX_CODE', 'PAT_STATE', 'PAT_ZIP', 'PROVIDER_ID', 'PROVIDER_NAME',
       'FIRST_PAYMENT_SRC', 'PAT_AGE', 'ADMIT_DAY_OF_WEEK', 'LENGTH_OF_STAY',
       'DISCHARGE_QTR', 'PRINC_DIAG_CODE', 'YEAR', 'QTR', 'CHARGES_PER_DAY',
       'PRINC_DIAG_DESC', 'PROVIDER_NAME_x', 'DISTANCE_TO_HOSPITAL',
       'IS_NEAREST', 'NEAREST_HOSPITAL_DISTANCE', 'NEAREST_HOSPITAL',
       'TRADEOFF', 'PVDR_LONG', 'PVDR_LAT', 'PAT_LONG', 'PAT_LAT'],
      dtype='object')

In [31]:
AllPatients_agg=AllPatients.groupby(['PROVIDER_ID',"PROVIDER_NAME",'PVDR_LONG','PVDR_LAT']).count()['DISCHARGE_QTR'].reset_index()
AllPatients_agg.columns=["PROVIDER_ID","PROVIDER_NAME","PVDR_LONG","PVDR_LAT","PATIENT_COUNT"]
project.save_data("HospitalTreatCondition.csv", AllPatients_agg.to_csv(index=False), overwrite=True)

{'file_name': 'HospitalTreatCondition.csv',
 'message': 'File saved to project storage.',
 'asset_id': '662d34a2-7c2d-48c8-aa24-bd56e4be5b79'}

***Next we can move to app-deployment notebook, where we will go through the steps to deploy the Shiny dashboard app.*** 